## 1. data_process 
- output: `{split}_data_processed.py`

- recommender 발화 기준 자르기 및 이전 utterance 누적

- data output format
    - "conv_id"
    - "contexts": []
        - role에 대한 speaker 추가 (e.g., "System:", "User:")
    - "response": 
    - "entities": []
    - "rec": []

In [2]:
dpath = "/home/hyuns6100/[4]newCRS/source/data/redial/"

import json
import os

entity2id = json.load(
            open(os.path.join(dpath, 'entity2id.json'), 'r', encoding='utf-8'))

In [1]:
import json
from tqdm import tqdm

def process(input_file, output_file, dpath):
    with open(dpath + input_file, 'r', encoding='utf-8') as fin, open(dpath + output_file, 'w', encoding='utf-8') as fout:
        # unique_conv_id = 0

        
        for line in tqdm(fin):
            conversation = json.loads(line)
            dialogs = conversation["dialog"]
            contexts = []
            resp = ''
            entities = []

            for i, message in enumerate(dialogs):
                if message['role'] == 'Seeker':
                    contexts.append(message['text'])
                    # entities.extend(message['entity'])
                    entity_ids = [entity2id[entity] for entity in message["entity"] if entity in entity2id]
                    movie_ids  = [entity2id[movie] for movie in message["movies"] if movie in entity2id]
                    entities.extend(entity_ids)
                    entities.extend(movie_ids)
                    
                elif message['role'] == 'Recommender':
                    # 첫 recommender 발화 또는 마지막 recommender 발화 뒤에 Seeker 발화가 없는 경우 무시
                    if i == 0 or i == len(conversation['dialog']) - 1 or conversation['dialog'][i + 1]['role'] != 'Seeker':
                        continue
                    
                
                # Update context and entities
                contexts.append(formatted_text)
                entity_ids = [entity2id[entity] for entity in message["entity"] if entity in entity2id]
                entities.extend(entity_ids)
                
                if message["role"] == "Seeker":
                    # # 다음 message가 대화의 마지막 message이면서 "role"이 "Recommender" 일 때
                    # if i + 1 == len(dialogs) - 1 and dialogs[i + 1]["role"] == "Recommender":
                    #     next_entities = [entity2id[entity] for entity in dialogs[i + 1]["entity"] if entity in entity2id]
                    #     entities.extend(next_entities)
                        
                    # Write the current context and entities
                    fout.write(json.dumps({
                        "conv_id": conversation["conv_id"],
                        "contexts": contexts,
                        "resp": # recommender의 발화
                        "entities": # 이전 발화에서 언급된 message["entity"] 리스트
                        "rec": # recommender의 발화에 언급된 message["moive"] 리스트 
                    }, ensure_ascii=False) + '\n')
                    
                    # Prepare for the next context
                    unique_conv_id += 1
                    #context = [formatted_text]  # Start new context with current Seeker's utterance
                    entities = [] # Reset entities for the new Seeker



        for i, message in enumerate(conversation['dialog']):
            # 이전 발화 누적
            if message['role'] == 'Seeker':
                contexts.append(message['text'])
                entities.extend(message['entity'])
            elif message['role'] == 'Recommender':
                # 첫 recommender 발화 무시
                if i == 0:
                    continue

                # 마지막 recommender 발화 뒤에 Seeker 발화가 없는 경우 무시
                if i == len(conversation['dialog']) - 1 or conversation['dialog'][i + 1]['role'] != 'Seeker':
                    continue

                # JSON 데이터 작성
                json_data = {
                    "conv_id": conversation["conv_id"],
                    "contexts": contexts.copy(),
                    "resp": message["text"],
                    "entities": entities.copy(),
                    "rec": message["movies"]
                }
                fout.write(json.dumps(json_data, ensure_ascii=False) + '\n')

                # 현재 recommender 발화를 contexts에 추가
                contexts.append(message['text'])
                entities.extend(message['entity'])



In [ ]:
dpath = "tmp_data/redial/"

entity2id = json.load(
            open(os.path.join(dpath, 'entity2id.json'), 'r', encoding='utf-8'))

process('senti_train_data_processed.jsonl', 'train_data_processed.jsonl', dpath)
process('senti_valid_data_processed.jsonl', 'valid_data_processed.jsonl', dpath)
process('senti_test_data_processed.jsonl', 'test_data_processed.jsonl', dpath)


In [51]:
import json

result = []

def save_conversation(data):
    global result
    
    contexts = []
    entities = []
    with open(data, 'r', encoding='utf-8') as fin:
        for line in tqdm(fin):
            
            conversation = json.loads(line)
            dialogs = conversation["dialog"]
            contexts = []
            resp = ''
            entities = []

            for i, message in enumerate(dialogs):
                # 이전 발화 누적
                if message['role'] == 'Seeker':
                    contexts.append(message['text'])
                    entity_ids = [entity2id[entity] for entity in message["entity"] if entity in entity2id]
                    movie_ids = [entity2id[movie] for movie in message["movies"] if movie in entity2id]
                    entities.extend(entity_ids)
                    entities.extend(movie_ids)
                
                # Recommender의 발화 처리
                elif message['role'] == 'Recommender':
                    if i == 0: 
                        contexts.append(message['text'])
                        entity_ids = [entity2id[entity] for entity in message["entity"] if entity in entity2id]
                        movie_ids = [entity2id[movie] for movie in message["movies"] if movie in entity2id]
                        entities.extend(entity_ids + movie_ids)
                        continue
                    
                    resp = message["text"]
                    resp_entity_ids = [entity2id[entity] for entity in message["entity"] if entity in entity2id] 
                    resp_movie_ids = [entity2id[movie] for movie in message["movies"] if movie in entity2id]
                    
                    json_data = {
                        "conv_id": conversation["conv_id"],
                        "contexts": contexts.copy(),
                        "resp": resp,
                        "entities": list(set(entities.copy())),
                        "rec": list(set(resp_movie_ids))
                    }
                    
                    result.append(json_data)
                    # fout.write(json.dumps(json_data, ensure_ascii=False) + '\n')

                    # 현재 recommender 발화를 contexts에 추가
                    contexts.append(message['text'])
                    entities.extend(list(set(resp_entity_ids + resp_movie_ids)))

                    # 마지막 대화가 Seeker의 발화인 경우 무시
                    if i == len(conversation['dialog']) - 2 and conversation['dialog'][i + 1]['role'] == 'Seeker':
                        continue

        
# 함수 호출
save_conversation(dpath + "senti_test_data_processed.jsonl")


1342it [00:00, 13581.69it/s]


In [52]:
result[21:25]

[{'conv_id': '20051',
  'contexts': ['What kind of movies do you like have you seen @205163 ? or @99896',
   'Yes and i liked them Those are awesome I like that kind of movies'],
  'resp': '@154513',
  'entities': [14589, 3278],
  'rec': [24689]},
 {'conv_id': '20051',
  'contexts': ['What kind of movies do you like have you seen @205163 ? or @99896',
   'Yes and i liked them Those are awesome I like that kind of movies',
   '@154513',
   'i also watched @99583'],
  'resp': 'those are good movies as well',
  'entities': [5977, 24689, 14589, 3278],
  'rec': []},
 {'conv_id': '20051',
  'contexts': ['What kind of movies do you like have you seen @205163 ? or @99896',
   'Yes and i liked them Those are awesome I like that kind of movies',
   '@154513',
   'i also watched @99583',
   'those are good movies as well',
   'No, i have not watched that one i think i will try that one'],
  'resp': '@100821 was another good xmen movie okay thanks!',
  'entities': [3278, 16655, 24689, 5977, 14589]

In [54]:
result[189:198]

[{'conv_id': '20303',
  'contexts': ['Hey how are you?',
   'Hey! I am great. How are you?',
   "I'm great thanks, what kind of movies do you like?",
   'I really love Disney movies. I love @107514 and @108278 .'],
  'resp': 'Awesome, have you tried the new @84854',
  'entities': [30176, 24077, 22686],
  'rec': [12418]},
 {'conv_id': '20303',
  'contexts': ['Hey how are you?',
   'Hey! I am great. How are you?',
   "I'm great thanks, what kind of movies do you like?",
   'I really love Disney movies. I love @107514 and @108278 .',
   'Awesome, have you tried the new @84854',
   "I haven't seen all of them though."],
  'resp': 'Those are great movies',
  'entities': [30176, 12418, 24077, 22686],
  'rec': []},
 {'conv_id': '20303',
  'contexts': ['Hey how are you?',
   'Hey! I am great. How are you?',
   "I'm great thanks, what kind of movies do you like?",
   'I really love Disney movies. I love @107514 and @108278 .',
   'Awesome, have you tried the new @84854',
   "I haven't seen all 